CAPÍTULO 02: PRÉ-PROCESSAMENTO DE DADOS

DESAFIO COMPUTACIONAL - CAPITULO 2 

O Capítulo 2 apresenta o exemplo do processo de preparação de base de dados aplicado
à base Mamo. Para este desafio, realize o mesmo processo para a base Bancos descrita
na Seção 2.1.2. Realize as etapas do processo de forma similar ao exemplo do livro

In [ ]:
import pandas as pd
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

## importação da base

In [ ]:


resp = urlopen('http://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip')
zipfile = ZipFile(BytesIO(resp.read()))
zipfile.namelist()
df = pd.read_csv(zipfile.open('bank-full.csv'), sep=';')

In [ ]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB


In [ ]:
df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


## Limpeza




In [ ]:
# verificação de dados ausentes
df.isna().sum() 
df.replace('unknown',np.nan, inplace=True)
df.replace('null',np.nan, inplace=True)
df.replace('nan',np.nan, inplace=True)
df.replace('NaN',np.nan, inplace=True)

In [ ]:
 df.isna().sum()

age              0
job            288
marital          0
education     1857
default          0
balance          0
housing          0
loan             0
contact      13020
day              0
month            0
duration         0
campaign         0
pdays            0
previous         0
poutcome     36959
y                0
dtype: int64

In [ ]:
# Remoção (não muito utilizada) 
# df.dropna(how='all', inplace=False)

# Inputação por variavel global
# df.replace(0,np.NAN, inplace=False)

#Inputação pela ultima observação
#ultima_observacao_default = df['default'].fillna(method='ffill')

#Inputação por modelo preditivo
#substitui os valores ausentes usando o valor mais frequente em cada coluna. Pode ser usado com strings ou dados numéricos.
#imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
#predict_loan = imp.fit_transform(df[['loan']])


#Inputação por moda
moda_job = df['job'].mode()
moda_education = df['education'].mode()

#Inputação pela moda de todos os objetos da mesma classe
moda_contact = df.groupby('y')['contact'].apply(lambda x: x.mode())
moda_contact
moda_education
moda_job


0    blue-collar
dtype: object

In [ ]:
valores_preenchimento = {
    'job': 'blue-collar', 
    'education': 'secondary',
    'contact': 'cellular'
}

In [ ]:
df.fillna(value=valores_preenchimento, inplace=True)
df.drop(columns=['poutcome'], inplace=True)

In [ ]:
df.isna().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
y            0
dtype: int64

## Discretização


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  y          45211 non-null  object
dtypes: int64(7), object(9)
memory usage: 5.5+ MB


In [ ]:
df['age'].value_counts()
bins_age = [30, 40, 50, 60, 70, 80, 90, 100]
df['age_disc'] = pd.cut(df['age'].to_numpy(), bins=bins_age, labels=[30, 40, 50, 60, 70, 80, 90], dtype)


In [ ]:
df['duration'].value_counts()
df['duration_disc'] = pd.cut(df['duration'].to_numpy(), bins=10, labels=[100, 200, 300, 400 , 500, 600, 700, 800, 900, 1000])

In [ ]:
df['balance'].value_counts()
df['balance_disc'] = pd.cut(df['balance'].to_numpy(), bins=5, labels=[1, 2, 3, 4, 5])

In [ ]:
df['pdays'].value_counts()
df['pdays_disc'] = pd.cut(df['pdays'].to_numpy(), bins=12, labels=[-1, 1, 100, 200, 300, 400 , 500, 600, 700, 800, 900, 1000])

## Tranformação dos dados
 


In [ ]:
## Busca dados dupliados
df.duplicated()
#df[df.duplicated()]

0        False
1        False
2        False
3        False
4        False
         ...  
45206    False
45207    False
45208    False
45209    False
45210    False
Length: 45211, dtype: bool

In [ ]:
df.tail()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,y,age_disc,duration_disc,balance_disc,pdays_disc
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,yes,50,200,1,-1
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,yes,70,100,1,-1
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,yes,70,300,1,100
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,no,50,200,1,-1
45210,37,entrepreneur,married,secondary,no,2971,no,no,cellular,17,nov,361,2,188,11,no,30,100,1,100



### normalização dos categoricos em texto por valores inteiros

In [ ]:
df['job'].value_counts()
df['job'].replace(0,0, inplace=True)
df['job'].replace('blue-collar',1, inplace=True)
df['job'].replace('management',2, inplace=True)
df['job'].replace('technician',3, inplace=True)
df['job'].replace('admin.',4, inplace=True)
df['job'].replace('services',5, inplace=True)
df['job'].replace('retired',6, inplace=True)
df['job'].replace('self-employed',7, inplace=True)
df['job'].replace('entrepreneur',8, inplace=True)
df['job'].replace('unemployed',9, inplace=True)
df['job'].replace('housemaid',10, inplace=True)
df['job'].replace('student',11, inplace=True)

df['marital'].replace('married',1, inplace=True)
df['marital'].replace('single',2, inplace=True)
df['marital'].replace('divorced',3, inplace=True)

df['education'].replace('primary',1, inplace=True)
df['education'].replace('secondary',2, inplace=True)
df['education'].replace('tertiary',3, inplace=True)
df['education'].replace('primary',1, inplace=True)

df['default'].replace('no', 0, inplace=True)
df['default'].replace('yes',1, inplace=True)

df['housing'].replace('no', 0, inplace=True)
df['housing'].replace('yes',1, inplace=True)

df['loan'].replace('no', 0, inplace=True)
df['loan'].replace('yes',1, inplace=True)

df['contact'].replace('cellular', 0, inplace=True)
df['contact'].replace('telephone',1, inplace=True)

df['month'].replace('jan', 1, inplace=True)
df['month'].replace('feb',2, inplace=True)
df['month'].replace('mar', 3, inplace=True)
df['month'].replace('apr',4, inplace=True)
df['month'].replace('may', 5, inplace=True)
df['month'].replace('jun',6, inplace=True)
df['month'].replace('jul', 7, inplace=True)
df['month'].replace('aug',8, inplace=True)
df['month'].replace('sep', 9, inplace=True)
df['month'].replace('oct',10, inplace=True)
df['month'].replace('nov', 11, inplace=True)
df['month'].replace('dec',12, inplace=True)

df['y'].replace('no', 0, inplace=True)
df['y'].replace('yes',1, inplace=True)

In [ ]:
df.tail()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,y,age_disc,duration_disc,balance_disc,pdays_disc
45206,51,3,1,3,0,825,0,0,0,17,11,977,3,-1,0,1,50,200,1,-1
45207,71,6,3,1,0,1729,0,0,0,17,11,456,2,-1,0,1,70,100,1,-1
45208,72,6,1,2,0,5715,0,0,0,17,11,1127,5,184,3,1,70,300,1,100
45209,57,1,1,2,0,668,0,0,1,17,11,508,4,-1,0,0,50,200,1,-1
45210,37,8,1,2,0,2971,0,0,0,17,11,361,2,188,11,0,30,100,1,100


In [ ]:
n

### normalização utilizando a função maxmin

In [ ]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   age            45211 non-null  int64   
 1   job            45211 non-null  int64   
 2   marital        45211 non-null  int64   
 3   education      45211 non-null  int64   
 4   default        45211 non-null  int64   
 5   balance        45211 non-null  int64   
 6   housing        45211 non-null  int64   
 7   loan           45211 non-null  int64   
 8   contact        45211 non-null  int64   
 9   day            45211 non-null  int64   
 10  month          45211 non-null  int64   
 11  duration       45211 non-null  int64   
 12  campaign       45211 non-null  int64   
 13  pdays          45211 non-null  int64   
 14  previous       45211 non-null  int64   
 15  y              45211 non-null  int64   
 16  age_disc       38181 non-null  category
 17  duration_disc  45211 non-null  

In [ ]:
df_normalize = normalize(df.drop(columns=['age_disc', 'duration_disc', 'balance_disc', 'pdays_disc']))
df_normalize['age_disc'] = df['age_disc']
df_normalize['duration_disc'] = df['duration_disc']
df_normalize['balance_disc'] = df['balance_disc']
df_normalize['pdays_disc'] = df['pdays_disc']
df_normalize.tail()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,y,age_disc,duration_disc,balance_disc,pdays_disc
45206,0.428571,0.2,0.0,1.0,0.0,0.080293,0.0,0.0,0.0,0.533333,0.909091,0.198658,0.032258,0.000000,0.000000,1.0,50,200,1,-1
45207,0.688312,0.5,1.0,0.0,0.0,0.088501,0.0,0.0,0.0,0.533333,0.909091,0.092721,0.016129,0.000000,0.000000,1.0,70,100,1,-1
45208,0.701299,0.5,0.0,0.5,0.0,0.124689,0.0,0.0,0.0,0.533333,0.909091,0.229158,0.064516,0.212156,0.010909,1.0,70,300,1,100
45209,0.506494,0.0,0.0,0.5,0.0,0.078868,0.0,0.0,1.0,0.533333,0.909091,0.103294,0.048387,0.000000,0.000000,0.0,50,200,1,-1
45210,0.246753,0.7,0.0,0.5,0.0,0.099777,0.0,0.0,0.0,0.533333,0.909091,0.073404,0.016129,0.216743,0.040000,0.0,30,100,1,100


### Redução de dados

In [ ]:
# Amostragem Aleatória Estratificada pela classe y

X_train, X_test, y_train, y_test = train_test_split(df_normalize.drop('y',axis=1),
df_normalize['y'], stratify=df_normalize['y'], test_size=0.20)

y_test.value_counts()


0.0    7985
1.0    1058
Name: y, dtype: int64